# Геолокация сообщений

#### Импорт библиотек

In [1]:
import warnings

warnings.simplefilter("ignore")

import os
import sys

folder = os.getcwd().split("\\examples")[0]
sys.path.append(folder)

import sloyka
from sloyka import Geocoder

INFO: Pandarallel will run on -1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


[nltk_data] Downloading package stopwords to /home/gk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Загрузка данных

In [2]:
import pandas as pd

df = pd.read_csv('/home/gk/Downloads/Telegram Desktop/ruina.csv', index_col=0)

In [3]:
df.head()

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack
0,2024-03-21 19:30:00,333766,В доме на Синопской набережной 32/35 из-за поз...,795.0,28,8.0,post,NaN,NaN,NaN
1,2024-03-21 10:30:00,333530,"Сделайте сад Сан-Галли безопасным для граждан,...",3527.0,126,23.0,post,NaN,NaN,NaN
2,2024-03-20 19:30:00,333376,«Электросамокатная саранча» вновь повылезала н...,9866.0,188,24.0,post,NaN,NaN,NaN
3,2024-03-20 10:30:00,333265,Вот и отгуляла широкая Масленица! Наша команда...,4599.0,54,8.0,post,"https://vk.com/wall-129354225_332885|ярко,",NaN,NaN
4,2024-03-19 19:30:00,333226,На том же месте сквозь года Вот так каланча ве...,4355.0,51,11.0,post,https://vk.com/wall-129354225_325408|уплотните...,NaN,NaN


#### Запуск геолокатора со встроенным геокодером

In [4]:
result = Geocoder().run(df, text_column='text')

2024-03-22 13:50:03,779 SequenceTagger predicts: Dictionary with 5 tags: O, S-ADDRESS, B-ADDRESS, E-ADDRESS, I-ADDRESS
03-22 13:50 | INFO     | clear_names started
03-22 13:50 | INFO     | get_street started
03-22 13:50 | INFO     | extract_ner_street started


100%|██████████| 720/720 [00:37<00:00, 19.45it/s]

03-22 13:51 | INFO     | pattern1.sub started



100%|██████████| 720/720 [00:00<00:00, 228797.55it/s]

03-22 13:51 | INFO     | pattern2.findall started


...

03-22 13:51 | INFO     | extract_toponym started



100%|██████████| 720/720 [00:00<00:00, 2038.75it/s]

03-22 13:51 | INFO     | get_stem started


03-22 13:52 | INFO     | create_gdf started


100%|██████████| 856/856 [02:08<00:00,  6.66it/s]


In [7]:
result

,date,id,text,views.count,likes.count,reposts.count,type,link,post_id,parents_stack,...,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,2024-03-21 19:30:00,333766,В доме на Синопской набережной 32/35 из-за поз...,795.0,28,8.0,post,NaN,NaN,NaN,...,0.991,,Синопской,None,Синопская набережная Санкт-Петербург Россия,['Синопская набережная Санкт-Петербург Россия'],Синопская набережная Санкт-Петербург Россия,Синопская набережная,"Синопская набережная, Пески, округ Смольнинско...",POINT (30.38832 59.92982)
1,2024-03-21 10:30:00,333530,"Сделайте сад Сан-Галли безопасным для граждан,...",3527.0,126,23.0,post,NaN,NaN,NaN,...,NaN,,Сан-Галли,None,None,None,None,None,None,None
2,2024-03-20 19:30:00,333376,«Электросамокатная саранча» вновь повылезала н...,9866.0,188,24.0,post,NaN,NaN,NaN,...,NaN,,None,None,,[''],,,None,None
3,2024-03-20 10:30:00,333265,Вот и отгуляла широкая Масленица! Наша команда...,4599.0,54,8.0,post,"https://vk.com/wall-129354225_332885|ярко,",NaN,NaN,...,0.973,78,,None,набережная канала Грибоедова 78 Санкт-Петербур...,['набережная канала Грибоедова 78 Санкт-Петерб...,набережная канала Грибоедова 78 Санкт-Петербур...,набережная канала Грибоедова,"набережная канала Грибоедова, Апраксин двор, о...",POINT (30.32656 59.93513)
4,2024-03-19 19:30:00,333226,На том же месте сквозь года Вот так каланча ве...,4355.0,51,11.0,post,https://vk.com/wall-129354225_325408|уплотните...,NaN,NaN,...,NaN,37,Загородном,None,Загородный проспект 37 Санкт-Петербург Россия,['Загородный проспект 37 Санкт-Петербург Россия'],Загородный проспект 37 Санкт-Петербург Россия,Загородный проспект,"Электродом, 37/70 литА, Загородный проспект, о...",POINT (30.33256 59.92240)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
851,2024-02-07 10:42:30,328420,"Люди десятилетиями ждут замены лифтов, чтоб он...",NaN,2,NaN,comment,NaN,328388.0,[],...,NaN,,None,None,,[''],,,None,None
852,2024-02-07 12:57:57,328422,"С вопросом капремонта лифтов, к сожалению, ст...",NaN,4,NaN,comment,NaN,328388.0,[],...,NaN,,None,None,,[''],,,None,None
853,2024-02-07 14:03:22,328427,Думаю что лифты в порядке. Просто их выключили...,NaN,1,NaN,comment,NaN,328388.0,[],...,NaN,,None,None,,[''],,,None,None
854,2024-02-07 15:26:01,328438,"[id10835085|Андрей], очень даже может быть",NaN,1,NaN,reply,NaN,328388.0,[328427],...,NaN,,None,None,,[''],,,None,None


In [6]:
result.to_file('ruina.geojson')

In [6]:
pd.set_option('display.max_colwidth', 20)
pd.reset_option('display.width', 10)
# pd.set_option('display.width', 2000)


In [7]:
mask = (result.Score.isna())&(~result.Street.isna())
result[mask]
# result.to_csv('C:\Users\USER\SOIKA\сохры.csv', index=False)

,message_id,Дата и время,Текст комментария,Street,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
9,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская у...,Адмиралтейская у...,Адмиралтейская у...,POINT (30.59423 ...
10,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская у...,Адмиралтейская н...,Адмиралтейская у...,POINT (30.59423 ...
11,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская н...,Адмиралтейская у...,Адмиралтейская н...,POINT (30.30745 ...
12,9,2023.01.13 20:58,а метро Адмиралт...,адмиралтейская,NaN,,Адмиралтейская,None,Адмиралтейская у...,['Адмиралтейская...,Адмиралтейская н...,Адмиралтейская н...,Адмиралтейская н...,POINT (30.30745 ...
13,10,2023.01.13 20:41,Б. Московская-ид...,социалистическая,NaN,,Социалистическая,улица,Социалистическая...,['Социалистическ...,Социалистическая...,Социалистическая...,Социалистическая...,POINT (30.13556 ...
74,49,2022.12.20 15:57,[id395057623|Све...,пушкинская,NaN,,Пушкинская,переулок,Пушкинская улица...,['Пушкинская ули...,Пушкинская улица...,Пушкинская улица,Пушкинская улица...,POINT (30.37686 ...
77,52,2022.12.20 10:27,Социалистическая...,загородного,NaN,,Загородного,None,Загородный просп...,['Загородный про...,Загородный просп...,Загородный проспект,Загородный просп...,POINT (30.33879 ...
87,62,2022.12.19 10:51,[id1209378|Ольга...,таврического сада,NaN,,Таврического сада,None,None,None,None,None,None,None
145,95,2022.12.14 22:26,К Гагаринской та...,гагаринской,NaN,,Гагаринской,None,Гагаринская улиц...,['Гагаринская ул...,Гагаринская улиц...,Гагаринская улица,Гагаринская улиц...,POINT (30.34291 ...
147,97,2022.12.14 11:25,[id186409804|Гал...,коломенская,NaN,,Коломенская,None,Коломенская улиц...,['Коломенская ул...,Коломенская улиц...,Коломенская улица,Коломенская улиц...,POINT (30.35294 ...


#### Визаулизация полученных сообщений

In [8]:
result.head(3)

,message_id,Дата и время,Текст комментария,Street,Score,Numbers,initial_street,Toponims,full_street_name,location_options,addr_to_geocode,only_full_street_name,Location,geometry
0,0,2023.01.26 16:32,Здравствуйте! В ...,мойка,0.997,11,Мойка,улица,набережная реки ...,['набережная рек...,набережная реки ...,набережная реки ...,"Business France,...",POINT (30.32437 ...
1,1,2023.01.26 11:55,[club143265175|Ц...,рубинштейна,0.900,2,Рубинштейна,None,улица Рубинштейн...,['улица Рубинште...,улица Рубинштейн...,улица Рубинштейна,"Parivar, 2/45, у...",POINT (30.34690 ...
2,2,2023.01.28 12:39,"1) Фурштатская, ...",фурштатская,0.784,19,Фурштатская,None,Фурштатская улиц...,['Фурштатская ул...,Фурштатская улиц...,Фурштатская улица,"Цветочки, 19, Фу...",POINT (30.35583 ...


In [9]:
# result.explore()